In [ ]:
import ast
import pathlib

In [ ]:
REPO_ROOT = pathlib.Path('../..').resolve()
LIBRARY_PATH = REPO_ROOT / 'lib'

CONVERSIONS = {
    "attr": "attrs",
    "PIL": "Pillow",
    "Image": "Pillow",
    "mpl_toolkits": "matplotlib",
    "dateutil": "python_dateutil",
    "skimage": "scikit-image",
    "yaml": "PyYAML",
    "plt": "matplotlib"
}

In [ ]:
def path_to_module(filepath, library_path):
    relative_path = filepath.relative_to(library_path)
    if relative_path.name == '__init__.py':
        relative_path = relative_path.parent
        
    module_name = '.'.join(relative_path.with_suffix('').parts)
    
    return module_name

In [ ]:
test_filepath = REPO_ROOT / 'lib' / 'pymedphys' / '_streamlit' / 'apps' / 'metersetmap' / 'main.py'

path_to_module(test_filepath, LIBRARY_PATH)

In [ ]:
all_filepaths = list(LIBRARY_PATH.glob("**/*.py"))
module_to_filepath_map = {
    path_to_module(filepath, LIBRARY_PATH): filepath
    for filepath in all_filepaths
}

# module_to_filepath_map

In [ ]:
def get_file_imports(filepath, library_path, conversions):
    relative_path = filepath.relative_to(library_path)
    
    with open(filepath, "r") as file:
        module_contents = file.read()

    parsed = ast.parse(module_contents)
    all_import_nodes = [
        node for node in ast.walk(parsed) 
        if isinstance(node, ast.Import) or isinstance(node, ast.ImportFrom)
    ]

    import_nodes = [
        node for node in all_import_nodes 
        if isinstance(node, ast.Import)
    ]
    import_from_nodes = [
        node for node in all_import_nodes 
        if isinstance(node, ast.ImportFrom)
    ]
    
    imports = set()

    for node in import_nodes:
        for alias in node.names:
            imports.add(alias.name)

    for node in import_from_nodes:
        if node.level == 0:
            if node.module.startswith('pymedphys._imports'):
                for alias in node.names:
                    try:
                        name = conversions[alias.name]
                    except KeyError:
                        name = alias.name
                    imports.add(name)
            else:
                for alias in node.names:
                    imports.add(f"{node.module}.{alias.name}")

        else:
            module = '.'.join(relative_path.parts[:-node.level])
            
            if node.module:
                module = f"{module}.{node.module}"
                
            for alias in node.names:
                imports.add(f"{module}.{alias.name}")
                
    return imports

In [ ]:
all_modules = set(module_to_filepath_map.keys())

In [ ]:
module_dependencies = {}

for module, filepath in module_to_filepath_map.items():
    raw_imports = get_file_imports(filepath, LIBRARY_PATH, CONVERSIONS)
    imports = set()
    for an_import in raw_imports:
        if an_import.startswith('pymedphys'):
            if an_import in all_modules:
                imports.add(an_import)
            else:
                adjusted_import = '.'.join(an_import.split('.')[:-1])
                if not adjusted_import in all_modules:
                    print(an_import)
                    print(adjusted_import)
                    raise ValueError()
                imports.add(adjusted_import)
                
        else:
            adjusted_import = an_import.split('.')[0]
            imports.add(adjusted_import)
            
    module_dependencies[module] = imports


In [ ]:
node = [
    node
    for node in ast.walk(ast.parse("from ._vendor.deprecated import deprecated as _deprecated"))
    if isinstance(node, (ast.ImportFrom, ast.Import))
][0]

name = node.names[0]
dir(name)

name.asname

In [ ]:
# all_modules

In [ ]:
module_dependencies